In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
from pycaret.time_series import *
from sktime.forecasting.model_selection import SlidingWindowSplitter
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
warnings.filterwarnings('ignore')

In [2]:
%pip install 'pycaret[full]'

  Using cached shap-0.44.1-cp311-cp311-macosx_11_0_arm64.whl.metadata (24 kB)
  Using cached interpret-0.5.1-py3-none-any.whl.metadata (1.1 kB)
  Using cached umap-learn-0.5.5.tar.gz (90 kB)
  Preparing metadata (setup.py) ... done
  Using cached PyYAML-5.3.1.tar.gz (269 kB)
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [51 lines of output]
      /opt/homebrew/lib/python3.11/site-packages/setuptools/config/setupcfg.py:293: _DeprecatedConfig: Deprecated config in `setup.cfg`
      !!
      
              ********************************************************************************
              The license_file parameter is deprecated, use license_files instead.
      
              This deprecation is overdue, please update your project and remove deprecated
              calls to avoid build errors in the future.
      
              See https://setuptools.pypa.io/en/l

In [7]:
%pip install 'packaging==21.3'

  Using cached packaging-21.3-py3-none-any.whl.metadata (15 kB)
Using cached packaging-21.3-py3-none-any.whl (40 kB)
  Attempting uninstall: packaging
    Found existing installation: packaging 23.2
    Uninstalling packaging-23.2:
      Successfully uninstalled packaging-23.2
Note: you may need to restart the kernel to use updated packages.


In [9]:
%pip install --upgrade 'packaging'
%pip install --upgrade 'pycaret'

  Using cached packaging-23.2-py3-none-any.whl.metadata (3.2 kB)
Using cached packaging-23.2-py3-none-any.whl (53 kB)
  Attempting uninstall: packaging
    Found existing installation: packaging 21.3
    Uninstalling packaging-21.3:
      Successfully uninstalled packaging-21.3
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
df = pd.read_csv("train.csv")
# df = df[['DATETIMEDATA','PM25']]
df = df.set_index('DATETIMEDATA')

df

,PM25,O3,WS,TEMP,RH,WD
DATETIMEDATA,,,,,,
2024-01-01 00:00:00,16.5,16.000000,1,27.0,79.0,115.0
2024-01-01 01:00:00,33.6,11.326513,1,27.0,82.0,97.0
2024-01-01 02:00:00,34.2,11.326513,1,26.0,84.0,96.0
2024-01-01 03:00:00,20.1,11.326513,1,26.0,82.0,108.0
2024-01-01 04:00:00,16.5,1.000000,1,27.0,79.0,106.0
...,...,...,...,...,...,...
2024-02-28 19:00:00,23.6,11.326513,1,29.0,64.0,184.0
2024-02-28 20:00:00,25.2,11.326513,1,29.0,64.0,183.0
2024-02-28 21:00:00,23.2,1.000000,1,29.0,64.0,159.0


In [3]:
import numpy as np
from sktime.forecasting.model_selection import SlidingWindowSplitter
ts = np.arange(10)
splitter = SlidingWindowSplitter(fh=np.arange(1,3), window_length=3, step_length=1)
list(splitter.split(ts))

[(array([0, 1, 2]), array([3, 4])),
 (array([1, 2, 3]), array([4, 5])),
 (array([2, 3, 4]), array([5, 6])),
 (array([3, 4, 5]), array([6, 7])),
 (array([4, 5, 6]), array([7, 8])),
 (array([5, 6, 7]), array([8, 9]))]

In [30]:
df.size

8496

In [4]:
exp = TSForecastingExperiment()
exp.setup(data = df, target='PM25' ,coverage=0.90, fold_strategy=SlidingWindowSplitter(fh=np.arange(1,23), window_length=130, step_length=130)) # using the past 6 months data to make prediction for the next month and moving half a year forward. 

TypeError: Unsupported input data type in ForecastingPipeline, input y must be in an sktime compatible format. Allowed scitypes for y in forecasting are Series, Panel, Hierarchical, for instance a pandas.DataFrame with sktime compatible time indices, or with MultiIndex and last(-1) level an sktime compatible time index. See the forecasting tutorial examples/01_forecasting.ipynb, or the data format tutorial examples/AA_datatypes_and_datasets.ipynb See the data format tutorial examples/AA_datatypes_and_datasets.ipynb. If you think the data is already in an sktime supported input format, run sktime.datatypes.check_raise(data, mtype) to diagnose the error, where mtype is the string of the type specification you want. Error message for checked mtypes, in format [mtype: message], as follows: [pd.DataFrame: y must be a pandas.DataFrame, found <class 'pandas.core.series.Series'>]  [pd.Series: <class 'pandas.core.indexes.base.Index'> is not supported for y, use one of (<class 'pandas.core.index

In [18]:
exp.plot_model(plot='train_test_split')


In [19]:
exp.compare_models(sort='rmse')


,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,20:30:54
Status,. . . . . . . . . . . . . . . . . .,Selecting Estimator
Estimator,. . . . . . . . . . . . . . . . . .,ARIMA


,Model,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2,TT (Sec)
huber_cds_dt,Huber w/ Cond. Deseasonalize & Detrending,0.9835,0.9591,3.9127,4.8513,0.2670,0.2452,-1.1621,0.1250
ridge_cds_dt,Ridge w/ Cond. Deseasonalize & Detrending,1.0297,0.9952,4.1509,5.1056,0.2883,0.2569,-1.7169,0.1280
auto_arima,Auto ARIMA,1.0706,1.0029,4.2582,5.1323,0.2974,0.2508,-1.5251,12.4080
et_cds_dt,Extra Trees w/ Cond. Deseasonalize & Detrending,1.0319,1.0066,4.1815,5.2213,0.3042,0.2470,-2.6226,0.2180
lasso_cds_dt,Lasso w/ Cond. Deseasonalize & Detrending,1.0784,1.0313,4.3321,5.2983,0.3191,0.2556,-2.8095,0.1170
llar_cds_dt,Lasso Least Angular Regressor w/ Cond. Deseasonalize & Detrending,1.0784,1.0313,4.3321,5.2983,0.3191,0.2556,-2.8095,0.1210
lr_cds_dt,Linear w/ Cond. Deseasonalize & Detrending,1.0604,1.0280,4.2998,5.3183,0.3035,0.3039,-2.5011,0.1550
en_cds_dt,Elastic Net w/ Cond. Deseasonalize & Detrending,1.0717,1.0373,4.3061,5.3193,0.3121,0.2545,-2.5254,0.1240
croston,Croston,1.1174,1.0393,4.5681,5.4240,0.3231,0.2680,-1.8330,0.0090
br_cds_dt,Bayesian Ridge w/ Cond. Deseasonalize & Detrending,1.0936,1.0555,4.4157,5.4448,0.3251,0.2579,-3.0726,0.1180


BaseCdsDtForecaster(fe_target_rr=[WindowSummarizer(lag_feature={'lag': [24, 23,
                                                                        22, 21,
                                                                        20, 19,
                                                                        18, 17,
                                                                        16, 15,
                                                                        14, 13,
                                                                        12, 11,
                                                                        10, 9,
                                                                        8, 7, 6,
                                                                        5, 4, 3,
                                                                        2, 1]},
                                                   n_jobs=1)],
                    regressor=HuberRegressor(), sp=24, window_length=24)

In [21]:
ets = exp.create_model('huber_cds_dt')


,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2
0,2024-01-06 09:00,0.9700,0.8222,3.3520,4.0181,0.1794,0.1821,0.1994
1,2024-01-11 19:00,1.1829,1.2907,3.5767,4.9467,0.1862,0.2134,-0.2009
2,2024-01-17 05:00,0.6915,0.7244,3.7239,4.7653,0.2792,0.2333,-1.8319
3,2024-01-22 15:00,1.3007,1.0940,6.1082,6.8222,0.6253,0.5281,-7.4278
4,2024-01-28 01:00,1.0868,1.1406,4.3861,5.4816,0.2573,0.2515,-1.4098
5,2024-02-02 11:00,1.1852,1.0998,4.7209,5.4531,0.2856,0.2423,-0.2207
6,2024-02-07 21:00,0.5043,0.4788,3.1982,3.8559,0.1665,0.1527,0.7040
7,2024-02-13 07:00,0.6782,0.6326,2.9022,3.6288,0.1656,0.1544,-0.2661
8,2024-02-18 17:00,1.3141,1.2583,3.8308,4.6793,0.2572,0.2301,0.0006
9,2024-02-24 03:00,0.9209,1.0492,3.3283,4.8626,0.2673,0.2636,-1.1677


In [22]:
tuned_ets = exp.tune_model(ets)

,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2
0,2024-01-06 09:00,1.0125,0.8535,3.4989,4.1713,0.1886,0.1897,0.1372
1,2024-01-11 19:00,1.2313,1.3854,3.7230,5.3096,0.1896,0.2244,-0.3836
2,2024-01-17 05:00,0.7254,0.7447,3.9065,4.8992,0.2913,0.2436,-1.9933
3,2024-01-22 15:00,1.1531,0.9788,5.4153,6.1039,0.5402,0.4662,-5.7466
4,2024-01-28 01:00,1.0623,1.1206,4.2873,5.3851,0.2517,0.2468,-1.3257
5,2024-02-02 11:00,1.1534,1.0476,4.5940,5.1942,0.2731,0.2378,-0.1075
6,2024-02-07 21:00,0.5338,0.5015,3.3853,4.0388,0.1762,0.1599,0.6752
7,2024-02-13 07:00,0.6773,0.6328,2.8983,3.6298,0.1654,0.1542,-0.2668
8,2024-02-18 17:00,1.3153,1.2601,3.8343,4.6860,0.2571,0.2306,-0.0023
9,2024-02-24 03:00,0.8138,0.8268,2.9411,3.8317,0.2301,0.2463,-0.3460


Fitting 10 folds for each of 10 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    4.0s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   10.3s finished


In [23]:
exp.plot_model(tuned_ets, plot='forecast', data_kwargs={'fh':36})


In [24]:
holdout_pred = exp.predict_model(tuned_ets, verbose=True)

,Model,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2
0,HuberRegressor,0.6830,0.7368,2.9758,4.2070,0.1406,0.1569,0.0222


In [25]:
holdout_pred

,y_pred
2024-02-28 02:00,13.9627
2024-02-28 03:00,13.1027
2024-02-28 04:00,14.8537
2024-02-28 05:00,17.0593
2024-02-28 06:00,17.4925
2024-02-28 07:00,18.9773
2024-02-28 08:00,19.0086
2024-02-28 09:00,17.6960
2024-02-28 10:00,17.3021
2024-02-28 11:00,16.5737


In [27]:
final_model = exp.finalize_model(tuned_ets)


In [28]:
exp.plot_model(final_model, plot='forecast', data_kwargs={'fh':168})


ValueError: Model was trained with exogenous variables but you have not passed any for predictions. Please pass exogenous variables to make predictions.

In [34]:
import pandas as pd
from statsmodels.tsa.statespace.sarimax import SARIMAX

# Assuming your data is stored in a DataFrame called df_train
endog = df['PM25']  # Endogenous variable (target variable)
exog = df[['O3', 'WS', 'TEMP', 'RH', 'WD']]  # Exogenous variables

# Define and train the SARIMAX model
model = SARIMAX(endog, exog=exog, order=(1, 0, 1), seasonal_order=(1, 1, 1, 24))
result = model.fit()

# Forecasting
forecast_horizon = 168  # Forecast horizon of 24 hours
forecast = result.forecast(steps=forecast_horizon, exog=exog.iloc[-forecast_horizon:])

# Extract the last datetime value from the train.csv DataFrame
last_datetime = df.index[-1]

# Create a new datetime index for the forecast DataFrame
forecast_index = pd.date_range(start=last_datetime, periods=forecast_horizon + 1, freq='H')[1:]

# Create a DataFrame with forecasted values and the new datetime index
forecast_df = pd.DataFrame({'DATETIMEDATA': forecast_index, 'PM25': forecast})

forecast_df

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =           10     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  2.71728D+00    |proj g|=  1.61845D-01


 This problem is unconstrained.



At iterate    5    f=  2.69496D+00    |proj g|=  2.33319D-01

At iterate   10    f=  2.63236D+00    |proj g|=  2.78912D-02

At iterate   15    f=  2.62549D+00    |proj g|=  6.10178D-02

At iterate   20    f=  2.61930D+00    |proj g|=  1.76111D-02

At iterate   25    f=  2.61897D+00    |proj g|=  1.33228D-02

At iterate   30    f=  2.61842D+00    |proj g|=  5.24540D-02

At iterate   35    f=  2.61819D+00    |proj g|=  1.82471D-03

At iterate   40    f=  2.61814D+00    |proj g|=  1.34068D-02

At iterate   45    f=  2.61810D+00    |proj g|=  8.03296D-03

At iterate   50    f=  2.61808D+00    |proj g|=  1.69998D-03

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tn

,DATETIMEDATA,PM25
2024-02-29 00:00:00,2024-02-29 00:00:00,19.766104
2024-02-29 01:00:00,2024-02-29 01:00:00,18.643713
2024-02-29 02:00:00,2024-02-29 02:00:00,18.218800
2024-02-29 03:00:00,2024-02-29 03:00:00,17.015873
2024-02-29 04:00:00,2024-02-29 04:00:00,18.705157
...,...,...
2024-03-06 19:00:00,2024-03-06 19:00:00,18.607291
2024-03-06 20:00:00,2024-03-06 20:00:00,18.714677
2024-03-06 21:00:00,2024-03-06 21:00:00,18.501272
2024-03-06 22:00:00,2024-03-06 22:00:00,18.972538


In [36]:
import pandas as pd
from statsmodels.tsa.statespace.sarimax import SARIMAX

# Load data from train.csv into a DataFrame
df = pd.read_csv('train.csv')

# Check the contents of the DataFrame df
print(df.head())

# Extract the last timestamp from the train.csv data
last_timestamp = df['DATETIMEDATA'].iloc[-1]
print("Last timestamp from train.csv:", last_timestamp)

# Generate a sequence of timestamps for the forecasted period
forecast_horizon = 24  # Forecast horizon of 24 hours
additional_steps = 168  # Number of additional time steps to predict beyond the initial forecast horizon (24*7)
forecast_dates = pd.date_range(start=last_timestamp, periods=forecast_horizon + additional_steps + 1, freq='H')[1:]

# Define endogenous and exogenous variables
endog = df['PM25']  # Endogenous variable (target variable)
exog = df[['O3', 'WS', 'TEMP', 'RH', 'WD']]  # Exogenous variables

# Define and train the SARIMAX model
model = SARIMAX(endog, exog=exog, order=(1, 0, 1), seasonal_order=(1, 1, 1, 24))
result = model.fit()

# Forecasting
extended_forecast = result.get_forecast(steps=forecast_horizon + additional_steps, exog=exog.iloc[-(forecast_horizon + additional_steps):])
predicted_values = extended_forecast.predicted_mean

# Create a DataFrame for the predicted values
predicted_df = pd.DataFrame({'DATETIMEDATA': forecast_dates, 'PM25': predicted_values})


predicted_df


          DATETIMEDATA  PM25         O3  WS  TEMP    RH     WD
0  2024-01-01 00:00:00  16.5  16.000000   1  27.0  79.0  115.0
1  2024-01-01 01:00:00  33.6  11.326513   1  27.0  82.0   97.0
2  2024-01-01 02:00:00  34.2  11.326513   1  26.0  84.0   96.0
3  2024-01-01 03:00:00  20.1  11.326513   1  26.0  82.0  108.0
4  2024-01-01 04:00:00  16.5   1.000000   1  27.0  79.0  106.0
Last timestamp from train.csv: 2024-02-28 23:00:00
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =           10     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  2.71728D+00    |proj g|=  1.61845D-01


 This problem is unconstrained.



At iterate    5    f=  2.69496D+00    |proj g|=  2.33319D-01

At iterate   10    f=  2.63236D+00    |proj g|=  2.78912D-02

At iterate   15    f=  2.62549D+00    |proj g|=  6.10178D-02

At iterate   20    f=  2.61930D+00    |proj g|=  1.76111D-02

At iterate   25    f=  2.61897D+00    |proj g|=  1.33228D-02

At iterate   30    f=  2.61842D+00    |proj g|=  5.24540D-02

At iterate   35    f=  2.61819D+00    |proj g|=  1.82471D-03

At iterate   40    f=  2.61814D+00    |proj g|=  1.34068D-02

At iterate   45    f=  2.61810D+00    |proj g|=  8.03296D-03

At iterate   50    f=  2.61808D+00    |proj g|=  1.69998D-03

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tn

,DATETIMEDATA,PM25_prediction
1416,2024-02-29 00:00:00,20.075080
1417,2024-02-29 01:00:00,18.998370
1418,2024-02-29 02:00:00,18.331131
1419,2024-02-29 03:00:00,16.703528
1420,2024-02-29 04:00:00,18.749276
...,...,...
1603,2024-03-07 19:00:00,18.607175
1604,2024-03-07 20:00:00,18.714532
1605,2024-03-07 21:00:00,18.501166
1606,2024-03-07 22:00:00,18.972411


In [38]:
#save to csv
predicted_df.to_csv('predict_hour.csv')

In [249]:
%pip install 'sktime'


77161.45s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
Note: you may need to restart the kernel to use updated packages.
